In [2]:
import sys
sys.path.insert(0, '../..')
sys.path.insert(0, 'plots')

import os

from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets


import json
from src.SplunkIntel import SplunkIntel
from src.sources.SplunkDataset import SplunkDataset
from src.sources.SplunkFileSource import SplunkFileSource

import plotly as py
import pandas as pd
import numpy as np
import plot as plot

py.offline.init_notebook_mode()
pd.set_option('display.notebook_repr_html', True)

data_sources_names = []

data_source_picker = None
time_range_picker = None
event_picker = None
event_text_area = None 
threshold_picker = None
new_time_range_picker = None

#create widgets
def create_widgets():
    global data_source_picker, time_range_picker, event_picker
    global event_text_area, threshold_picker, new_time_range_picker
    for file in os.listdir("../../data"):
            if file.endswith(".json"):
                data_sources_names.append(file)

    data_source_picker = widgets.Dropdown(
            options=data_sources_names,
            description='Data Source:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    

    time_range_picker = widgets.IntRangeSlider(
                value=[1, 2],
                min=0,
                max=100,
                step=1,
                description='Control Time:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='i',
                slider_color='white',
                color='black'
        )
    
    new_time_range_picker = widgets.IntRangeSlider(
                value=[1, 2],
                min=0,
                max=100,
                step=1,
                description='Test Time:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='i',
                slider_color='white',
                color='black'
        )
    
    threshold_picker = widgets.FloatSlider(
                value=0.5,
                min=0,
                max=1,
                step=0.1,
                description='Threshold:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
                slider_color='white',
                color='black'
        )
    
    event_picker = widgets.Dropdown(
            options=[1,2,3],
            description='Event Number:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )
    
    l = widgets.Layout(height='40px', width='800px')
    event_text_area = widgets.Textarea(value='TA: height=40px', layout=l)

    
def initialize(change):    
    # Just to set the widgets
    all_events = []
    all_events.extend(SplunkFileSource.load_data('../../data/' + data_source_picker.value))    
    minutes = 0
    for dict in all_events:
        if dict.get('cluster_label') == '1':
            minutes = minutes + 1

    time_range_picker.value = [1,2]
    time_range_picker.min = 1
    time_range_picker.max = minutes
    
    new_time_range_picker.value = [minutes -2, minutes]
    new_time_range_picker.min = 1
    new_time_range_picker.max = minutes 
    
    change_event_handler(None)
    
    
splunkDataset = None
splunkIntelargs = []

def event_picker_change_handler(change):
    event_text_area.value = json.dumps(splunkDataset.get_all_events()[event_picker.value])

def split(input, length, size):
    input.replace('\n', ' ')
    input.replace('\tat', ' ')
    return '<br>'.join([input[start:start + size] for start in range(0, length, size)])

def get_labels(control_events, anomalies):
    labels = []
    for i in range(len(control_events)):
        labels.append(0)
    for p in anomalies:
        if p == 1:
            labels.append(1)
        else:
            labels.append(2)

    return labels

def get_tool_tips(control_events, test_events, control_clusters, test_clusters):
    tooltips = []
    for idx, event in enumerate(control_events):
        tooltips.append(
            split(event[0], min(1000, len(event[0])), 100) + '<br> id = ' + str(event[3]) + '<br> cluster = ' + str(
            control_clusters[idx]))

    for idx, event in enumerate(test_events):
        tooltips.append(
            split(event[0], min(1000, len(event[0])), 100) + '<br> id = ' + str(event[3]) + '<br> cluster = ' + str(
            test_clusters[idx]))
            
    return tooltips
    
# Handle data source change
def change_event_handler(change):
    global splunkDataset
    
    clear_output()
    
    splunkIntelargs.append('--file_source')
    splunkIntelargs.append('../../data/' + data_source_picker.value) 
    splunkIntelargs.append('--control_window')
    splunkIntelargs.append(str(int(time_range_picker.value[0])))
    splunkIntelargs.append(str(int(time_range_picker.value[1])))
    splunkIntelargs.append('--test_window')
    splunkIntelargs.append(str(int(new_time_range_picker.value[0])))
    splunkIntelargs.append(str(int(new_time_range_picker.value[1])))
    splunkIntelargs.append('--sim_threshold')
    splunkIntelargs.append(str(float(threshold_picker.value)))
    
    print(splunkIntelargs)

    splunkIntel = SplunkIntel(SplunkIntel.parse(splunkIntelargs))
    splunkDataset = splunkIntel.run_from_file()

    
    plot.scatter_plot(splunkDataset.get_xy_matrix_control(), splunkDataset.get_control_events())

    plot.scatter_plot_groups(splunkDataset.get_xy_matrix_control(), 
                             splunkDataset.get_control_clusters(), 
                             splunkDataset.get_cluster_tags(5))
    
    plot.scatter_plot_groups(splunkDataset.get_xy_matrix_all(), 
                             get_labels(splunkDataset.get_control_events(), splunkDataset.get_anomalies()),
                            get_tool_tips(splunkDataset.get_control_events(), splunkDataset.get_test_events()
                                         , splunkDataset.get_control_clusters(),splunkDataset.get_test_clusters()), 
                             ['control', 'test', 'test-anomaly'], ['blue','orange','red'])
    
    plot.scatter_plot_groups_4d(splunkDataset.get_xyz_matrix_all(), 
                                get_labels(splunkDataset.get_control_events(), splunkDataset.get_anomalous_values()), 
                                splunkDataset.get_all_clusters(),
                                get_tool_tips(splunkDataset.get_control_events(), 
                                              splunkDataset.get_test_events(),
                                              splunkDataset.get_control_clusters(),
                                              splunkDataset.get_test_clusters()),
                                ['blue','orange','red'])


            
    event_picker.options = [ x for x in range(len(splunkDataset.get_all_events()))] 
     

create_widgets()

#intialize
initialize(None)
    
#setup even handlers    
data_source_picker.observe(initialize, names="value")
time_range_picker.observe(change_event_handler, names="value")
new_time_range_picker.observe(change_event_handler, names="value")
event_picker.observe(event_picker_change_handler, names="value")
threshold_picker.observe(change_event_handler, names="value")



# show widgets
display(widgets.HBox([data_source_picker]))
display(widgets.HBox([time_range_picker, threshold_picker]))
display(widgets.HBox([new_time_range_picker]))
display(widgets.HBox([event_picker, event_text_area]))

plotting....


clustering....


Detecting new anomaly


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()